In [1]:
#####ベイジアン変量効果二項回帰モデル#####
##みどり本10章:個体差と生存種子数
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [8]:
####データの発生####
#np.random.seed(8742)   #シードを設定

#データの設定
N = 1000   #観測個体数
n = poisson(12.5, N)   #個体ごとの種子数

In [9]:
##応答変数の生成
#パラメータの設定
alpha = alphat = normal(0, 0.4, N)   #変量効果
beta = betat = -0.5   #全体共通の切片

#ロジットモデルと確率を設定
logit = alpha + beta   #ロジット
Pr = np.exp(logit) / (1 + np.exp(logit))   #確率

#二項分布から応答変数を生成
y = binomial(n, Pr, N)
pd.DataFrame({'1. 種子数' : n, '2. 生存数' : y, '3. 観測確率' : y/n, '4. 真の確率' : Pr})   #データの確認

,1. 種子数,2. 生存数,3. 観測確率,4. 真の確率
0,4,2,0.500000,0.190283
1,7,4,0.571429,0.376966
2,12,5,0.416667,0.230502
3,17,7,0.411765,0.528030
4,14,5,0.357143,0.292969
5,11,2,0.181818,0.376984
6,14,4,0.285714,0.349588
7,10,5,0.500000,0.596146
8,13,4,0.307692,0.367232
9,18,5,0.277778,0.209161


In [11]:
####マルコフ連鎖モンテカルロ法で変量効果二項回帰モデルを推定####
##二項回帰モデルの対数尤度関数を設定
def define_likelihood(alpha, beta, n, y):
    #ロジットと確率の定義
    logit = alpha + beta
    Pr = np.exp(logit)/(1+np.exp(logit))
    
    #対数尤度の計算
    LLi = y * np.log(Pr) + (n-y) * np.log(1-Pr)
    LL = sum(LLi)   
    return LL, LLi

In [12]:
##アルゴリズムの設定
R = 10000
keep = 4  
iter = 0
burnin = 2000/keep
disp = 100

##事前分布の設定
tau1 = 100   #パラメータの事前分布
s0 = 0.01   #階層モデルの事前分布
v0 = 0.01
rw1 = 0.1   #ランダムウォーク幅
rw2 = 0.15

##初期値の設定
alpha = normal(0, 0.2, N)
beta = 0
tau2 = 0.5   #階層モデルの分散の初期値

##パラメータの保存用配列
ALPHA = np.zeros((int(R/keep), N))
BETA = np.zeros((int(R/keep)))
LL = np.zeros((int(R/keep)))

In [13]:
####メトロポリスヘイスティング法でパラメータをサンプリング####
for rp in range(R):
    ##全体共通のbetaをサンプリング
    #betaをサンプリング
    betad = beta
    betan = betad + normal(0, rw1)

    #対数尤度と対数事前分布の計算
    lognew1 = define_likelihood(alpha, betan, n, y)
    logold1 = define_likelihood(alpha, betad, n, y)
    logpnew1 = -0.5 * (np.power(betan, 2) / 2*tau1)
    logpold1 = -0.5 * (np.power(betad, 2) / 2*tau1)

    #MHサンプリングでbetaを採択するかどうかを決定
    gamma = min(1, np.exp(lognew1[0] + logpnew1 - logold1[0] - logpold1))
    u = uniform(0, 1, 1)

    #alpha > u なら新しいbetaを採択
    if gamma > u:
        beta = betan
        logl = lognew1
    else :
        beta = betad
        logl = logold1


    ##変量効果alphaをサンプリング
    #alphaをサンプリング
    alphad = alpha
    alphan = alphad + normal(0, rw2, N)

    #対数尤度と対数事前分布の計算
    lognew2 = define_likelihood(alphan, beta, n, y)
    logold2 = logl
    logpnew2 = -0.5 * (np.power(alphan, 2) / 2*tau2)
    logpold2 = -0.5 * (np.power(alphad, 2) / 2*tau2)

    #MHサンプリングでbetaを採択するかどうかを決定
    rand = uniform(0, 1, N)   #一様分布から乱数を発生
    LLind_diff = np.exp(lognew2[1] + logpnew2 - logold2[1] - logpold2)   #採択率
    gamma = (LLind_diff > 1)*1 + (LLind_diff <= 1)*LLind_diff

    #gammaの値に基づき新しいalphaを採択
    flag = (gamma >= rand)*1 + (gamma < rand)*0
    alpha = flag*alphan + (1-flag)*alphad


    ##逆ガンマ分布から階層モデルの分散をサンプリング
    s = s0 + sum(np.power(alpha - np.mean(alpha), 2))   
    v = v0 + N 
    tau2 = 1/np.random.gamma(v/2, 1/(s/2), 1)   #逆ガンマ分布からtauをサンプリング
    
    ##サンプリング結果の保存
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[mkeep] = beta
        ALPHA[mkeep, :] = alpha
        LL[mkeep] = logl[0]
    
    if rp%disp==0:
        print(rp)
        print(np.array([beta, tau2, ]))
        print(logl[0])

0
[-0.0672486   0.05054847]
-8503.201010626466
100
[-0.48389062  0.67641743]
-8054.399175776737
200
[-0.56997035  0.83412395]
-8012.5614214313855
300
[-0.56299454  0.86468729]
-8022.3348987654335
400
[-0.51980981  0.90193922]
-8028.996087431288
500
[-0.52289238  0.83768178]
-8015.026897396204
600
[-0.47550844  0.89482282]
-7998.703935714583
700
[-0.46864427  0.80491016]
-8022.110407956724
800
[-0.44603125  0.89537186]
-7997.612021040476
900
[-0.52141558  0.85265348]
-8024.91989564152
1000
[-0.52087091  0.88258769]
-7985.241233236118
1100
[-0.41089571  0.89177375]
-7991.698416007566
1200
[-0.45917654  0.79307949]
-8022.6857393300415
1300
[-0.39476459  0.83136363]
-7969.205146998805
1400
[-0.4154856   0.85597103]
-8019.551657881699
1500
[-0.46983246  0.83869415]
-7998.528906946586
1600
[-0.4332677  0.8595201]
-7980.587265844132
1700
[-0.43591983  0.73743734]
-7983.209301032499
1800
[-0.4269053   0.76837198]
-8016.924216484378
1900
[-0.40392269  0.84083969]
-8014.679912913516
2000
[-0.474